In [94]:
company = input("Enter the company name: ")

# Financial Ratios

In [95]:
import requests
from bs4 import BeautifulSoup
import sqlite3

def create_database(company):
    # Connect to SQLite database
    conn = sqlite3.connect(f'{company}.db')
    cursor = conn.cursor()

    # Create table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS company_data (
                        id INTEGER PRIMARY KEY,
                        field TEXT,
                        value TEXT
                    )''')
    conn.commit()

    # Close connection
    conn.close()

def insert_data(company, data):
    # Connect to SQLite database
    conn = sqlite3.connect(f'{company}.db')
    cursor = conn.cursor()

    # Insert or replace data into SQLite database
    for field, value in data.items():
        cursor.execute("SELECT id FROM company_data WHERE field=?", (field,))
        existing_record = cursor.fetchone()
        if existing_record:
            cursor.execute("UPDATE company_data SET value=? WHERE field=?", (value, field))
        else:
            cursor.execute("INSERT INTO company_data (field, value) VALUES (?, ?)", (field, value))
        conn.commit()

    # Close connection
    conn.close()

def scrape_data(url):
    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the grid with id 'top-ratios'
    top_ratios_grid = soup.find('ul', {'id': 'top-ratios'})

    # Extracting data from the grid
    data = {}
    for li in top_ratios_grid.find_all('li'):
        name = li.find('span', class_='name').get_text(strip=True)
        value = li.find('span', class_='number').get_text(strip=True)
        data[name] = value

    return data

if __name__ == "__main__":
    
    url = f'https://www.screener.in/company/{company}/consolidated/'
    create_database(company)
    scraped_data = scrape_data(url)
    insert_data(company, scraped_data)
    print("Data inserted into SQLite database successfully!")


Data inserted into SQLite database successfully!


# Quarterly Financial Data

In [96]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_and_return_tables(company, url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        divs = soup.find_all('div', class_='responsive-holder fill-card-width')
        tables = {}
        for i, div in enumerate(divs, 1):
            table = div.find('table')
            if table:
                header_row = table.find('tr')
                column_names = [header.text.strip() for header in header_row.find_all('th')]
                data = []
                for row in table.find_all('tr')[1:]:
                    row_data = [col.get_text(strip=True) for col in row.find_all('td')]
                    data.append(row_data)
                df = pd.DataFrame(data, columns=column_names)
                if i == 1:
                    globals()['quarterly_results'] = df
                elif i == 2:
                    globals()['yearly_results'] = df
                elif i == 3:
                    globals()['balancesheet'] = df
                elif i == 4:
                    globals()['cashflows'] = df
                elif i == 5:
                    globals()['ratios'] = df
                elif i == 6:
                    globals()['shareholding_quarterly'] = df
                elif i == 7:
                    globals()['shareholding_yearly'] = df
                else:
                    print(f"Extra table found, ignoring.")
        return tables
    else:
        print("Failed to retrieve data from the URL.")
        return None

if __name__ == "__main__":
    url = f'https://www.screener.in/company/{company}/consolidated/'
    tables = scrape_and_return_tables(company, url)
    if tables:
        for table_name, df in tables.items():
            print(f"{table_name}:")
            print(df)
            print("=" * 50)

import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

tables = [
    ('quarterly_results', "TCS"),
    ('yearly_results', 'TCS'),
    ('balancesheet', 'TCS'),
    ('cashflows', 'TCS'),
    ('ratios', 'TCS'),
    ('shareholding_quarterly', 'TCS'),
    ('shareholding_yearly', 'TCS')
]

def create_database():
    conn = sqlite3.connect('TCS.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS company_data (
                        id INTEGER PRIMARY KEY,
                        company TEXT,
                        field TEXT,
                        value TEXT
                    )''')
    conn.commit()
    conn.close()

def insert_dataframe_to_sqlite(df, company,financial_data):
    if df is not None:
        df.rename(columns={'': 'Category'}, inplace=True)
        conn = sqlite3.connect(f'{company}.db')
        try:
            df.to_sql(financial_data, conn, if_exists='replace', index=False)
            conn.commit()
        finally:
            conn.close()

if __name__ == "__main__":  
    
    # Create a list of dataframe names
    df_names = ['quarterly_results', 'yearly_results', 'balancesheet', 'cashflows', 'ratios', 'shareholding_quarterly', 'shareholding_yearly']
    financial_data_names = ['quarterly_data', 'yearly_data', 'balancesheet_data', 'cashflows_data', 'ratios_data', 'shareholding_quarterly_data', 'shareholding_yearly_data']
    # Iterate over the list and print each dataframe
    for df_name, financial_data_name in zip(df_names, financial_data_names):
        df = globals()[df_name]  # Get the dataframe object using its name
        insert_dataframe_to_sqlite(df, company, financial_data_name)




In [25]:
import yfinance as yf
import plotly.graph_objs as go
from datetime import datetime, timedelta

def plot_stock_price(symbol, duration):
    # Determine the start date based on the duration
    if duration == '1yr':
        start_date = datetime.now() - timedelta(days=365)
    elif duration == '2yr':
        start_date = datetime.now() - timedelta(days=365*2)
    elif duration == '5yr':
        start_date = datetime.now() - timedelta(days=365*5)
    else:
        print("Invalid duration. Please choose from '1yr', '2yr', or '5yr'.")
        return

    # Fetch historical stock data for the specified duration
    stock_data = yf.download(symbol, start=start_date, end=datetime.now())

    # Create a trace for the closing prices
    trace = go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines', name=symbol)

    # Create layout
    layout = go.Layout(title=f'Stock Price Chart for {symbol} ({duration})',
                       xaxis=dict(title='Date'),
                       yaxis=dict(title='Price (USD)'),
                       hovermode='closest',
                       showlegend=True)

    # Create figure
    fig = go.Figure(data=[trace], layout=layout)

    # Show plot
    fig.show()

# Example usage: Plotting the stock price for Apple Inc. (AAPL) for 2 years
plot_stock_price('AAPL', '2yr')


[*********************100%%**********************]  1 of 1 completed
